Загружаем данные

In [ ]:
import requests

with open('Lab1.zip', 'wb') as fh:
  resp = requests.get('https://github.com/DimaKurd/cv_lab1/raw/master/lab1/Lab1.zip')
  fh.write(resp.content)

In [ ]:
! unzip Lab1.zip -d .

Archive:  Lab1.zip
   creating: ./images/
  inflating: ./images/Abyssinian_122.jpg  
  inflating: ./images/Abyssinian_19.jpg  
  inflating: ./images/Abyssinian_210.jpg  
  inflating: ./images/Abyssinian_24.jpg  
  inflating: ./images/Abyssinian_31.jpg  
  inflating: ./images/Abyssinian_44.jpg  
  inflating: ./images/Abyssinian_66.jpg  
  inflating: ./images/Abyssinian_9.jpg  
  inflating: ./images/american_bulldog_140.jpg  
  inflating: ./images/american_bulldog_199.jpg  
  inflating: ./images/american_bulldog_95.jpg  
  inflating: ./images/basset_hound_118.jpg  
  inflating: ./images/basset_hound_14.jpg  
  inflating: ./images/basset_hound_181.jpg  
  inflating: ./images/beagle_1.jpg   
  inflating: ./images/beagle_180.jpg  
  inflating: ./images/Bengal_103.jpg  
  inflating: ./images/Bengal_147.jpg  
  inflating: ./images/Bengal_50.jpg  
  inflating: ./images/Birman_31.jpg  
  inflating: ./images/Birman_7.jpg   
  inflating: ./images/Bombay_16.jpg  
  inflating: ./images/Bombay_48.jp

In [ ]:
import os

import numpy as np
import pandas as pd
import cv2

from scipy import signal
from tqdm import tqdm
from typing import List

import warnings
warnings.filterwarnings("ignore")

Метод для обработки изображения

In [70]:
class CannyEdgeDetector (object):
    def get_img_edges(self, img, sigma, low_threshold, high_threshold):
        grad_magnitute, grad_degree = self.get_mag_degree(img)
        supressed = self.non_maximal_supress(grad_magnitute, grad_degree)
        thresholded = self.double_threshold(supressed, low_threshold, high_threshold)
        output = self.edge_tracking(thresholded)
        
        return output
    
    @staticmethod
    def non_maximal_supress(img, gdegree):
        height, width = img.shape
    
        for x in range(0, width):
            for y in range(0, height):
                if x == 0 or y == height -1 or y == 0 or x == width -1:
                    img[y][x] = 0
                    continue
                direction = gdegree[y][x] % 4
                if direction == 0:
                    if img[y][x] <= img[y][x - 1] or img[y][x] <= img[y][x + 1]:
                        img[y][x] = 0
                if direction == 1:
                    if img[y][x] <= img[y - 1][x + 1] or img[y][x] <= img[y + 1][x - 1]:
                        img[y][x] = 0
                if direction == 2:
                    if img[y][x] <= img[y - 1][x] or img[y][x] <= img[y + 1][x]:
                        img[y][x] = 0
                if direction == 3:
                    if img[y][x] <= img[y - 1][x - 1] or img[y][x] <= img[y + 1][x + 1]:
                        img[y][x] = 0

        return img

    @staticmethod
    def double_threshold(img, low_threshold, high_threshold):
        img[np.where(img > high_threshold)] = 0
        img[np.where((img >= low_threshold) & (img <= high_threshold))] = 255
        img[np.where(img < low_threshold)] = 0

        return img

    @staticmethod
    def edge_tracking(img):
        height, width = img.shape

        for i in range(0, height):
            for j in range(0, width):
                if img[i][j] == 75:
                    if ((img[i + 1][j] == 255) or 
                        (img[i - 1][j] == 255) or 
                        (img[i][j + 1] == 255) or 
                        (img[i][j - 1] == 255) or 
                        (img[i + 1][j + 1] == 255) or 
                        (img[i - 1][j - 1] == 255)):
                        img[i][j] = 255
                    else:
                        img[i][j] = 0

        return img

    @staticmethod
    def get_mag_degree(img):
        Lx = np.array([[1, 0, -1],
                    [2, 0, -2],
                    [1, 0, -1]])
        Ly = np.array([[1, 2, 1],
                    [0, 0, 0],
                    [-1, -2, -1]])
        G = np.array([[2, 4, 5, 4, 2],
                    [4, 9, 12, 9, 4],
                    [5, 12, 15, 12, 5],
                    [4, 9, 12, 9, 4],
                    [2, 4, 5, 4, 2]])
        G = G / 159
        G_x = signal.convolve2d(G, Lx, mode='same')
        G_y = signal.convolve2d(G, Ly, mode='same')
        magx = signal.convolve2d(img, G_x, mode='same')
        magy = signal.convolve2d(img, G_y, mode='same')

        mag = np.sqrt(magx**2 + magy**2)
        degree = np.arctan2(magy, magx)

        return mag, degree
    

In [74]:
def get_foreground_mask(image_path: str) -> List[tuple]:
    """
    Метод для вычисления маски переднего плана на фото
    :param image_path - путь до фото
    :return массив в формате [(x_1, y_1), (x_2, y_2), (x_3, y_3)], в котором перечислены все точки, относящиеся к маске
    """

    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    canny = CannyEdgeDetector()
    img_edges = canny.get_img_edges(img, 1, 25, 80)

    img = np.where(img_edges > 1, 255, 0).astype('uint8')
    pred_points = np.argwhere(img)
    pred_points[:, [1, 0]] = pred_points[:, [0, 1]]
    cv2.fillPoly(img, [pred_points], 255)
    pred_points = np.argwhere(img)

    return pred_points


Расчёт метрики

In [72]:
def evaluate_iou(image_dir: str, anno_dir: str) -> float:
    """
    Метод для расчёта mean IoU на датасете
    :param image_dir - каталог с фото для анализа
    :param anno_dir - каталог с аннотациями для расчета метрики
    :return значение mean IoU с сохранением оценки по каждому фото в csv-файл
    """

    iou_data = dict()
    for _, _, files in os.walk(image_dir):
        assert files is not None, 'no files read'
        for image_name in tqdm(files):
            # print(f'Processing file {image_name}')
            sample_name = image_name[:image_name.find('.jpg')]

            # acquiring ground truth mask data
            mask_true = cv2.imread(os.path.join(anno_dir, f'{sample_name}.png'))
            assert mask_true is not None, 'mask is None'
            true_points = cv2.cvtColor(mask_true, cv2.COLOR_BGR2GRAY)
            true_points[true_points != 2] = 1
            true_points[true_points == 2] = 0
            true_points = np.argwhere(true_points)
            true_points_set = set([tuple(x) for x in true_points])

            # acquiring predicted mask
            pred_points = get_foreground_mask(image_path=os.path.join(image_dir, image_name))
            assert pred_points is not None, 'pred_points is None'
            pred_points_set = set([tuple(x) for x in pred_points])

            # calculating IoU
            iou = len(true_points_set.intersection(pred_points_set)) / len(true_points_set.union(pred_points_set))

            image_names = iou_data.get('image_names', [])
            image_names.append(sample_name)
            iou_data['image_names'] = image_names

            iou_values = iou_data.get('iou_values', [])
            iou_values.append(iou)
            iou_data['iou_values'] = iou_values

        pd.DataFrame(data=iou_data).to_csv('detailed_results.csv')
        return np.mean(iou_data['iou_values'])


Вызов расчёта метрики

In [73]:
print(f"Metric_value {evaluate_iou(image_dir='./images', anno_dir='./annotations')}")

Processing file boxer_11.jpg
Processing file Abyssinian_24.jpg
Processing file Russian_Blue_171.jpg
Processing file Abyssinian_210.jpg
Processing file beagle_180.jpg
Processing file basset_hound_181.jpg
Processing file Egyptian_Mau_95.jpg
Processing file Siamese_128.jpg
Processing file pug_13.jpg
Processing file Bengal_147.jpg
Processing file Sphynx_139.jpg
Processing file basset_hound_14.jpg
Processing file Maine_Coon_79.jpg
Processing file keeshond_92.jpg
Processing file Abyssinian_66.jpg
Processing file leonberger_100.jpg
Processing file yorkshire_terrier_90.jpg
Processing file Birman_7.jpg
Processing file shiba_inu_132.jpg
Processing file Persian_16.jpg
Processing file Ragdoll_188.jpg
Processing file chihuahua_38.jpg
Processing file english_setter_96.jpg
Processing file Maine_Coon_3.jpg
Processing file Russian_Blue_24.jpg
Processing file Abyssinian_19.jpg
Processing file shiba_inu_72.jpg
Processing file British_Shorthair_186.jpg
Processing file Sphynx_33.jpg
Processing file english